In [20]:
import json
import os
import pandas as pd

In [1]:
data = '/data_alfred/'

In [6]:
explainer_res_p = '/home/hoyeung/blob_alfred_data/exp_all/model:seq2seq_per_subgoal,name:v2_epoch_40_obj_instance_enc_max_pool_dec_aux_loss_weighted_bce_1to2/valid_unseen_argmax.debug_epoch_32.preds.json'
baseline_res_p = '/home/hoyeung/blob_alfred_data/exp_all/model:seq2seq_per_subgoal,name:v2_epoch_35_baseline/valid_unseen.debug_epoch_29.preds.json'

with open(explainer_res_p, 'r') as f:
    explainer_res = json.load(f)

with open(baseline_res_p, 'r') as f:
    baseline_res = json.load(f)

assert len(explainer_res) == len(baseline_res) == 254

keys = list(explainer_res.keys())

In [7]:
def get_human_annotation(root):
    root = root.replace('/root', '')
    with open(os.path.join(root, 'pp', 'ann_0.json'), 'r') as f:
        traj_data = json.load(f)
    return traj_data['turk_annotations']['anns'][0]['high_descs']


In [43]:

#  [ { id: , human: [] , model_type: "explainer", generation: []} ]

output = []

id_col = []
task_name_col = []
video_url_col = []
human_col = []
model_type_col = []
generation_col = []


for key in keys:

    h_ann = get_human_annotation(explainer_res[key]['root'])
    num_subgoal = len(h_ann)
    explainer_instr = []
    baseline_instr = []

    for subgoal_i in range(num_subgoal):
        explainer_instr.append(explainer_res[key]['p_lang_instr'][str(subgoal_i)])
        baseline_instr.append(baseline_res[key]['p_lang_instr'][str(subgoal_i)])

    task_name = explainer_res[key]['root'].split('/')[-2]
    # https://mturk.jessethomason.com/lang_2_plan/2.1.0/pick_cool_then_place_in_recep-LettuceSliced-None-DiningTable-17/trial_T20190909_070538_437648/video.mp4
    url = 'https://mturk.jessethomason.com/lang_2_plan/2.1.0/{}/{}/video.mp4'.format(task_name, key[:-2])

    # to json
    output.append({'id': '{}_{}'.format(key[:-2], 'explainer'), 'task_name':task_name, 'video_url':url, 'human':h_ann, 'model_type':'explainer',  'generation':explainer_instr})
    output.append({'id': '{}_{}'.format(key[:-2], 'baseline'), 'task_name':task_name, 'video_url':url, 'human':h_ann, 'model_type':'baseline',  'generation':baseline_instr})

    # to csv
    id_col.append('{}_{}'.format(key[:-2], 'explainer')); id_col.append('{}_{}'.format(key[:-2], 'baseline'))
    task_name_col.append(task_name); task_name_col.append(task_name)
    video_url_col.append(url); video_url_col.append(url)
    human_col.append(h_ann); human_col.append(h_ann)
    model_type_col.append('explainer'); model_type_col.append('baseline')
    generation_col.append(explainer_instr); generation_col.append(baseline_instr)

assert len(id_col) == len(task_name_col) == len(video_url_col) == len(human_col) == len(model_type_col) == len(generation_col)
    

In [44]:
# save out json
with open(os.path.join('/home/hoyeung', 'human_eval', 'instructions_20200817.json'), 'w') as f:
    json.dump(output, f)

In [45]:
# save out csv
csv_output = {
    'id': id_col,
    'task_name': task_name_col,
    'video_url': video_url_col,
    'human': human_col,
    'model_type': model_type_col,
    'generation': generation_col
    }

df = pd.DataFrame(csv_output, columns= ['id', 'task_name', 'video_url', 'human', 'model_type', 'generation'])
df.to_csv(os.path.join('/home/hoyeung', 'human_eval', 'instructions_20200817.csv'))


## Ignore (examine results)

In [40]:
df

,id,task_name,video_url,human,model_type,generation
0,trial_T20190908_025158_582174_explainer,pick_heat_then_place_in_recep-BreadSliced-None...,https://mturk.jessethomason.com/lang_2_plan/2....,"[turn around, go to your left to the counter t...",explainer,"[turn right and walk to the end of the room , ..."
1,trial_T20190908_025158_582174_baseline,pick_heat_then_place_in_recep-BreadSliced-None...,https://mturk.jessethomason.com/lang_2_plan/2....,"[turn around, go to your left to the counter t...",baseline,[turn around and go to the counter on the left...
2,trial_T20190909_123918_983775_explainer,pick_heat_then_place_in_recep-PotatoSliced-Non...,https://mturk.jessethomason.com/lang_2_plan/2....,"[move around to the table, pick up a knife fro...",explainer,[turn right and walk to the counter then turn ...
3,trial_T20190909_123918_983775_baseline,pick_heat_then_place_in_recep-PotatoSliced-Non...,https://mturk.jessethomason.com/lang_2_plan/2....,"[move around to the table, pick up a knife fro...",baseline,"[turn around and walk to the end of the room ,..."
4,trial_T20190907_074556_124850_explainer,pick_and_place_simple-Watch-None-Safe-219,https://mturk.jessethomason.com/lang_2_plan/2....,"[Turn right, walk to the cabinet, Open the fir...",explainer,"[turn around and walk to the dresser ., open t..."
5,trial_T20190907_074556_124850_baseline,pick_and_place_simple-Watch-None-Safe-219,https://mturk.jessethomason.com/lang_2_plan/2....,"[Turn right, walk to the cabinet, Open the fir...",baseline,"[turn around and walk to the counter ., take t..."
6,trial_T20190907_074124_966890_explainer,pick_clean_then_place_in_recep-SoapBar-None-Co...,https://mturk.jessethomason.com/lang_2_plan/2....,"[Move to the counter in between the two sinks,...",explainer,"[turn around and walk to the sink ., pick up t..."
7,trial_T20190907_074124_966890_baseline,pick_clean_then_place_in_recep-SoapBar-None-Co...,https://mturk.jessethomason.com/lang_2_plan/2....,"[Move to the counter in between the two sinks,...",baseline,[turn around and walk to the sink on the right...
8,trial_T20190908_113523_123938_explainer,pick_heat_then_place_in_recep-Egg-None-Garbage...,https://mturk.jessethomason.com/lang_2_plan/2....,[turn around and walk to the end of the kitche...,explainer,[turn around and walk to the small black bin i...
9,trial_T20190908_113523_123938_baseline,pick_heat_then_place_in_recep-Egg-None-Garbage...,https://mturk.jessethomason.com/lang_2_plan/2....,[turn around and walk to the end of the kitche...,baseline,[turn around and go to the right side of the t...


In [17]:
output[0]

{'generation': ['turn right and walk to the end of the room , then turn left and walk to the counter .',
  'pick up a slice of bread from the counter .',
  'turn to the right to face the counter .',
  'place the bread on the counter .',
  'pick up the knife on the counter .',
  'slice the bread on the counter .',
  'put the knife on the counter',
  'pick up a slice of bread from the counter .',
  'turn right and walk to the microwave above the stove .',
  'heat the bread in the microwave .',
  'turn right and walk to the counter .',
  'put the bread on the counter in front of the coffee maker .'],
 'human': ['turn around, go to your left to the counter that is to the right of the white fridge',
  'pick up the loaf of bread that is on the very rear of the counter',
  'place it to the front of the counter',
  'pick up the knife with the yellow handle',
  'cut the bread into several slices',
  'continue cutting the bread',
  'place the knife back on the counter next to the lettuce',
  'pi

In [18]:
output[1]


{'generation': ['turn around and go to the counter on the left side of the stove .',
  'pick up a slice of bread .',
  'put the bread back on the counter .',
  'place the bread on the counter next to the apple .',
  'pick up the bread from the counter .',
  'slice the bread on the counter .',
  'place the knife on the counter in front of the bread .',
  'pick up the bread from the counter .',
  'turn right and walk to the wall then turn left and face the microwave .',
  'open the microwave door , place the bread inside , turn on the microwave , after a couple seconds open the microwave back up and remove the now some bread , then close the microwave door .',
  'turn to the right and walk to the window .',
  'put the lettuce on the counter in front of the coffee machine .'],
 'human': ['turn around, go to your left to the counter that is to the right of the white fridge',
  'pick up the loaf of bread that is on the very rear of the counter',
  'place it to the front of the counter',
  '